In [10]:
import torch
import math
import torch.nn as nn
from torch.nn import functional as F
from dataclasses import dataclass

In [11]:
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()

        self.n_head = config.n_head
        self.n_embd = config.n_embd

        assert config.n_embd % config.n_head == 0

        # key, query and value projections for all heads but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)

        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)

        

        self.register_buffer('bias', torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size)) # not to be considered model params, does not require gradients

    def forward(self, x):

        B, T, C = x.size() # batch_size, number of tokens/ time step, n_embd (channels)

        # c = n_head * head_size, head_size determines dimensions of each k, q and v vectors

        qkv = self.c_attn(x) # compute a combined output vector for k, q and v
        q, k, v = qkv.split(self.n_embd, dim=2) # split the combined k, q and v into individual vectors 

        # reshaping k, q and v for parallel attention
        k = k.view(B, T, self.n_head, C//self.n_head).transpose(1, 2) # (B, n_head, T, head_size)
        q = q.view(B, T, self.n_head, C//self.n_head).transpose(1, 2) # (B, n_head, T, head_size)
        v = v.view(B, T, self.n_head, C//self.n_head).transpose(1, 2) # (B, n_head, T, head_size)

        att = (q @ k.transpose(-2, -1) * (1.0 / math.sqrt(k.size(-1)))) # performs Q*K^T / root(d_K)

        att = att.masked_fill(self.bias[:,:,:T,:T]==0, float('-inf')) # so that tokens only attend to past tokens never the future tokens

        att = F.softmax(att, dim=1)

        y = att @ v # similar to doing a weighted sum of the values of tokens that are interesting

        y = y.transpose(1, 2).contiguous().view(B, T, C) # combining the outputs of all heads maintaining the original embeddings structure

        #output projection   
        y = self.c_proj(y)

        return y


In [12]:
@dataclass
class GPTConfig:

    block_size:int = 1024 # maximum length of the input tokens 
    vocab_size:int = 50257 # number of tokens, will change for our tokenizer
    n_layer:int = 12 # number of layers
    n_head: int = 12 # number of attention heads
    n_embd:int = 768 # dimensions of the embeddings

In [13]:
# feedforward layer
class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd) # fully connected layer to project the input data to a higher dimension
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd) # projection layer to project the input back down to the original dimension

    def forward(self, x):

        x = self.c_fc(x)
        x = self.gelu(x)
        x= self.c_proj(x)

        return x

In [14]:
# layers of the transformer
class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):

        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))

        return x

In [15]:
class GPT(nn.Module):

    def __init__(self, config) :
        super().__init__()

        self.config = config

        self.transformer = nn.ModuleDict(dict(

            wte = nn.Embedding(config.vocab_size, config.n_embd), # token embeddings
            wpe = nn.Embedding(config.block_size, config.n_embd), # positonal encodings
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]), # stacking the layers, each block corresponds to a single layer of the transformer
            ln_f = nn.LayerNorm(config.n_embd), # layer normalization

        ))

        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias = False) # output layer, the output vector represents raw logits

    def forward(self, idx):

        # idx is of shape (B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
        # forward the token and posisition embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (B, T, n_embd)
        x = tok_emb + pos_emb

        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
            
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        return logits


    @classmethod
    def from_pretrained(cls, model_type):

        """loads pre-trained gpt2 weights from hugging face"""

        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}

        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]

        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints

        # create a from-scratch initialized GPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):


                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:

                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


In [16]:
# autodetect devide
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'

print(f'Using device : {device}')
    

Using device : mps


In [17]:
# output generaton
num_return_sequnces = 5
max_length = 30

model = GPT.from_pretrained('gpt2')

model.eval()
model.to(device)

loading weights from pretrained gpt: gpt2


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [18]:
# prefix tokens

# ---- replace this with our tokenizer ------
import tiktoken

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode('write the latex code for printing x^2 / 34')
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequnces, 1)
x = tokens.to(device)
# ----------------------

torch.manual_seed(42)

# torch.cuda.manual_seed(42)



while x.size(1) < max_length :

    with torch.no_grad() :

        logits = model(x)

        # logits = logits[0]

        print(logits.shape)
        
        logits = logits[:, -1, :]

        probs = F.softmax(logits, dim = -1)

        # get the top 50 probabilities only
        topk_probs, topk_indices = torch.topk(probs, 50, dim = -1)

        ix = torch.multinomial(topk_probs, 1)
        
        xcol = torch.gather(topk_indices, -1, ix)

        x = torch.cat((x,xcol), dim = 1)

# print text
for i in range(num_return_sequnces):

    tokens = x[i, : max_length].tolist()
    decoded = enc.decode(tokens)

    print('>', decoded)


torch.Size([5, 11, 50257])
torch.Size([5, 12, 50257])
torch.Size([5, 13, 50257])
torch.Size([5, 14, 50257])
torch.Size([5, 15, 50257])
torch.Size([5, 16, 50257])
torch.Size([5, 17, 50257])
torch.Size([5, 18, 50257])
torch.Size([5, 19, 50257])
torch.Size([5, 20, 50257])
torch.Size([5, 21, 50257])
torch.Size([5, 22, 50257])
torch.Size([5, 23, 50257])
torch.Size([5, 24, 50257])
torch.Size([5, 25, 50257])
torch.Size([5, 26, 50257])
torch.Size([5, 27, 50257])
torch.Size([5, 28, 50257])
torch.Size([5, 29, 50257])
> write the latex code for printing x^2 / 34!!!!!!!!!!!!!!!!!!!
> write the latex code for printing x^2 / 34!!!!!!!!!!!!!!!!!!!
> write the latex code for printing x^2 / 34!!!!!!!!!!!!!!!!!!!
> write the latex code for printing x^2 / 34!!!!!!!!!!!!!!!!!!!
> write the latex code for printing x^2 / 34!!!!!!!!!!!!!!!!!!!
